In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Iterable, Set, Tuple
import pandas as pd
from typing import Iterable, Dict, Optional, Tuple, List, Set
import geopandas as gpd
import networkx as nx
import os

In [4]:
TSV_PATH = "data/gadm1_nuts3_counties-gadm1_nuts3_counties - FB Social Connectedness Index - October 2021.tsv"
MAP_PATH = "data/gadm1_nuts3_counties_levels.csv"

In [46]:
# ================================
# Funzioni di utilità (semplici)
# ================================
from typing import Iterable, Dict, Optional, Tuple
import pandas as pd

def _normalize_location_code(s: pd.Series) -> pd.Series:
    """
    Normalizza i codici location: stringa, strip, upper.
    Non modifica il NaN.
    """
    return (
        s.astype("string")
         .str.strip()
         .str.upper()
    )

def _ensure_columns(df: pd.DataFrame, required: Iterable[str], where: str = "") -> None:
    """
    Verifica che il DataFrame contenga le colonne richieste.
    Lancia un'eccezione con messaggio chiaro in caso manchi qualcosa.
    """
    missing = [c for c in required if c not in df.columns]
    if missing:
        loc = f" in {where}" if where else ""
        raise ValueError(f"Mancano colonne{loc}: {missing}")

def _preview(df: pd.DataFrame, name: str = "DataFrame", n: int = 5) -> None:
    """
    Stampa anteprima, schema e numero righe.
    """
    print(f"== {name} – prime righe ==")
    display(df.head(n))
    print("\nSchema:")
    print(df.dtypes)
    print(f"\nNumero righe: {len(df):,}")

# ===========================================
# Lettura SCI (Facebook) e file di mapping
# ===========================================
def load_sci_tsv(
    path: str,
    sci_cols: Iterable[str] = ("user_loc", "fr_loc", "scaled_sci"),
    dtype_map: Optional[Dict[str, str]] = None,
    low_memory: bool = False
) -> pd.DataFrame:
    """
    Legge il TSV del Social Connectedness Index (SCI) di Facebook.
    - Se dtype_map è None, usa dtype ragionevoli di default.
    - Normalizza i codici (user_loc, fr_loc).
    """
    if dtype_map is None:
        dtype_map = {"user_loc": "string", "fr_loc": "string", "scaled_sci": "float64"}

    df = pd.read_csv(
        path,
        sep="\t",
        usecols=list(sci_cols),
        dtype=dtype_map,
        low_memory=low_memory
    )

    # Normalizzazione codici location
    if "user_loc" in df.columns:
        df["user_loc"] = _normalize_location_code(df["user_loc"])
    if "fr_loc" in df.columns:
        df["fr_loc"] = _normalize_location_code(df["fr_loc"])

    # Controllo colonne richieste
    _ensure_columns(df, sci_cols, where="SCI TSV")

    return df


def load_levels_mapping(
    path: str,
    usecols: Iterable[str] = ("key", "level"),
    rename_map: Dict[str, str] = {"key": "location_code", "level": "level_type"},
) -> pd.DataFrame:
    """
    Legge il file di mapping (gadm1_nuts3_counties_levels.csv).
    Rinomina colonne in: location_code, level_type.
    Normalizza location_code.
    """
    df = pd.read_csv(
        path,
        usecols=list(usecols),
        dtype="string"
    ).rename(columns=rename_map)

    # Controllo colonne richieste dopo rinomina
    _ensure_columns(df, ["location_code", "level_type"], where="Mapping livelli")

    # Normalizzazione codice
    df["location_code"] = _normalize_location_code(df["location_code"])

    return df


def quick_read_inputs(
    tsv_path: str,
    map_path: str,
    sci_cols: Iterable[str] = ("user_loc", "fr_loc", "scaled_sci"),
    dtype_map: Optional[Dict[str, str]] = None
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Wrapper comodo: legge entrambi i file e fa una preview.
    Ritorna (df_sci, df_map).
    """
    df_sci = load_sci_tsv(tsv_path, sci_cols=sci_cols, dtype_map=dtype_map)
    _preview(df_sci, name="SCI (TSV)")

    df_map = load_levels_mapping(map_path)
    _preview(df_map, name="Mapping livelli")

    return df_sci, df_map


In [11]:
# ===========================================
# Copertura: nodi, archi, (opz.) per paese
# ===========================================
from typing import Dict, Iterable, Optional, Tuple
import pandas as pd

# --- riuso delle utility già definite ---
def _normalize_location_code(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip().str.upper()

def _ensure_columns(df: pd.DataFrame, required: Iterable[str], where: str = "") -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        loc = f" in {where}" if where else ""
        raise ValueError(f"Mancano colonne{loc}: {missing}")

# -----------------------------------------------------------------------------
# Estrazione insiemi di codici e mappati
# -----------------------------------------------------------------------------
def get_unique_location_codes(df_sci: pd.DataFrame) -> pd.Series:
    """
    Ritorna tutti i codici unici presenti in user_loc ∪ fr_loc (normalizzati).
    """
    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")
    u = _normalize_location_code(df_sci["user_loc"])
    v = _normalize_location_code(df_sci["fr_loc"])
    return pd.Index(u).append(pd.Index(v)).astype("string").unique()

def get_mapped_codes(df_map: pd.DataFrame) -> pd.Series:
    """
    Ritorna i codici location_code presenti nel mapping (normalizzati).
    """
    _ensure_columns(df_map, ["location_code"], where="Mapping livelli")
    return _normalize_location_code(df_map["location_code"]).dropna().unique()

# -----------------------------------------------------------------------------
# Copertura nodi
# -----------------------------------------------------------------------------
def compute_node_coverage(df_sci: pd.DataFrame, df_map: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, float]]:
    """
    Calcola copertura nodi:
      - totale codici (SCI)
      - codici mappati (presenti in df_map)
      - codici non mappati (lista)
    Ritorna (df_unmapped, summary_dict)
    """
    sci_codes = pd.Series(get_unique_location_codes(df_sci), name="location_code")
    map_codes = pd.Series(get_mapped_codes(df_map), name="location_code")

    total_codes = sci_codes.size
    mapped_mask = sci_codes.isin(set(map_codes))
    mapped_count = int(mapped_mask.sum())
    unmapped = sci_codes[~mapped_mask].to_frame().drop_duplicates().sort_values("location_code").reset_index(drop=True)

    summary = {
        "total_unique_codes": int(total_codes),
        "mapped_unique_codes": int(mapped_count),
        "unmapped_unique_codes": int(total_codes - mapped_count),
        "node_coverage_pct": (mapped_count / total_codes * 100.0) if total_codes else 0.0,
    }
    return unmapped, summary

# -----------------------------------------------------------------------------
# Copertura archi
# -----------------------------------------------------------------------------
def compute_edge_coverage(df_sci: pd.DataFrame, df_map: pd.DataFrame) -> Dict[str, float]:
    """
    Copertura archi: quante righe SCI hanno BOTH endpoints mappati.
    """
    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")
    mapped_set = set(get_mapped_codes(df_map))

    u = _normalize_location_code(df_sci["user_loc"])
    v = _normalize_location_code(df_sci["fr_loc"])

    both_mapped_mask = u.isin(mapped_set) & v.isin(mapped_set)
    total_rows = int(len(df_sci))
    valid_rows = int(both_mapped_mask.sum())

    return {
        "total_rows": total_rows,
        "valid_rows_both_mapped": valid_rows,
        "edge_coverage_pct": (valid_rows / total_rows * 100.0) if total_rows else 0.0,
    }

# -----------------------------------------------------------------------------
# Copertura per paese (richiede country_ISO3 nel mapping)
# -----------------------------------------------------------------------------
def compute_country_coverage(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    iso_col: str = "country_ISO3",
    keep_only_intra_country: bool = True
) -> Optional[pd.DataFrame]:
    """
    Calcola copertura per paese:
      - nodi unici mappati che appaiono in archi validi
      - archi validi (righe SCI con entrambi i capi mappati)
      - opzionalmente solo archi intra-country (ISO uguali)
    Se df_map non contiene iso_col → ritorna None e stampa un avviso.
    """
    if iso_col not in df_map.columns:
        print(f"[AVVISO] Colonna '{iso_col}' assente nel mapping: copertura per paese non calcolata.")
        return None

    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")
    # mapping code -> ISO
    df_map_local = df_map[["location_code", iso_col]].copy()
    df_map_local["location_code"] = _normalize_location_code(df_map_local["location_code"])

    code2iso = df_map_local.dropna().drop_duplicates("location_code").set_index("location_code")[iso_col]

    # aggiunge ISO a partire dai codici
    sci = df_sci[["user_loc", "fr_loc"]].copy()
    sci["user_loc"] = _normalize_location_code(sci["user_loc"])
    sci["fr_loc"]   = _normalize_location_code(sci["fr_loc"])

    sci["iso_from"] = sci["user_loc"].map(code2iso)
    sci["iso_to"]   = sci["fr_loc"].map(code2iso)

    # tiene solo righe con entrambi mappati
    sci_valid = sci.dropna(subset=["iso_from", "iso_to"]).copy()

    if keep_only_intra_country:
        sci_valid = sci_valid[sci_valid["iso_from"] == sci_valid["iso_to"]].copy()

    # archi per paese (conteggio righe)
    edges_by_country = (sci_valid
                        .groupby("iso_from", as_index=False)
                        .agg(edges=("user_loc", "size"))
                        .rename(columns={"iso_from": iso_col}))

    # nodi per paese (nodi che compaiono almeno in un arco valido)
    nodes_from = sci_valid[["iso_from", "user_loc"]].rename(columns={"iso_from": iso_col, "user_loc": "loc"})
    nodes_to   = sci_valid[["iso_to", "fr_loc"]].rename(columns={"iso_to": iso_col, "fr_loc": "loc"})
    nodes_all  = pd.concat([nodes_from, nodes_to], ignore_index=True).drop_duplicates()

    nodes_by_country = (nodes_all.groupby(iso_col, as_index=False)
                        .agg(nodes=("loc", "nunique")))

    # merge nodi + archi e percentuale sul totale archi
    country_cov = edges_by_country.merge(nodes_by_country, on=iso_col, how="outer").fillna(0)
    country_cov["edges"] = country_cov["edges"].astype(int)
    country_cov["nodes"] = country_cov["nodes"].astype(int)

    total_edges = int(country_cov["edges"].sum()) if len(country_cov) else 0
    if total_edges > 0:
        country_cov["edges_pct"] = 100.0 * country_cov["edges"] / total_edges
    else:
        country_cov["edges_pct"] = 0.0

    # ordina per importanza (più archi, poi più nodi)
    country_cov = country_cov.sort_values(["edges", "nodes"], ascending=False).reset_index(drop=True)
    return country_cov

# -----------------------------------------------------------------------------
# Wrapper pratico: stampa un mini-report
# -----------------------------------------------------------------------------
def coverage_report(df_sci: pd.DataFrame, df_map: pd.DataFrame, top_n: int = 10) -> None:
    # nodi
    unmapped_nodes, node_summary = compute_node_coverage(df_sci, df_map)
    print("== Copertura NODI ==")
    for k, v in node_summary.items():
        print(f"- {k}: {v}")
    if len(unmapped_nodes) > 0:
        print(f"\nEsempi codici NON mappati ({min(10, len(unmapped_nodes))}):")
        display(unmapped_nodes.head(10))

    # archi
    edge_summary = compute_edge_coverage(df_sci, df_map)
    print("\n== Copertura ARCHI ==")
    for k, v in edge_summary.items():
        print(f"- {k}: {v}")

    # per paese (se disponibile)
    country_cov = compute_country_coverage(df_sci, df_map)
    if country_cov is not None and len(country_cov):
        print("\n== Copertura PER PAESE (prime righe) ==")
        display(country_cov.head(top_n))
    else:
        print("\n== Copertura PER PAESE ==")
        print("Mapping senza 'country_ISO3' → se vuoi questa sezione, aggiungi la colonna a df_map.")


In [9]:
sci_cols = ["user_loc", "fr_loc", "scaled_sci"]
dtype_map = {"user_loc": "string", "fr_loc": "string", "scaled_sci": "float64"}

In [10]:
df_sci, df_map = quick_read_inputs(
    tsv_path=TSV_PATH,
    map_path=MAP_PATH,
    sci_cols=sci_cols,
    dtype_map=dtype_map
)

== SCI (TSV) – prime righe ==


,user_loc,fr_loc,scaled_sci
0,ABW,ABW,11264841.0
1,ABW,AGO1,38.0
2,ABW,AGO10,34.0
3,ABW,AGO11,32.0
4,ABW,AGO12,23.0



Schema:
user_loc      string[python]
fr_loc        string[python]
scaled_sci           float64
dtype: object

Numero righe: 63,824,121
== Mapping livelli – prime righe ==


,location_code,level_type
0,AND,country
1,ATG,country
2,ABW,country
3,BHS,country
4,BRB,country



Schema:
location_code    string[python]
level_type       string[python]
dtype: object

Numero righe: 8,008


In [20]:
df_map['level_type'].value_counts()

level_type
COUNTY     3229
GADM1      1839
NUTS3      1522
GADM2      1370
COUNTRY      48
Name: count, dtype: Int64

In [47]:
# =========================================================
# NODES BUILDER (NUTS3, GADM2, US COUNTIES) – funzioni base
# =========================================================
from typing import Iterable, Dict, Optional, Tuple, List, Set
import pandas as pd
import geopandas as gpd

# ---------- utility già viste ----------
def _normalize_location_code(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip().str.upper()

def _ensure_columns(df: pd.DataFrame, required: Iterable[str], where: str = "") -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        loc = f" in {where}" if where else ""
        raise ValueError(f"Mancano colonne{loc}: {missing}")

# ---------- 0) Target codes: presenti in SCI e nel mapping, per tipo ----------
def select_target_codes(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    level_types: Iterable[str] = ("NUTS3", "GADM2", "COUNTY")
) -> pd.DataFrame:
    """
    Ritorna un df (location_code, level_type) con SOLO i codici:
      - presenti in df_sci (in user_loc o fr_loc)
      - presenti nel mapping df_map
      - il cui level_type è tra quelli richiesti
    """
    _ensure_columns(df_sci, ["user_loc", "fr_loc"], "SCI")
    _ensure_columns(df_map, ["location_code", "level_type"], "Mapping")

    sci_codes = pd.Index(_normalize_location_code(df_sci["user_loc"])) \
                  .append(pd.Index(_normalize_location_code(df_sci["fr_loc"]))) \
                  .unique()

    df_map2 = df_map.copy()
    df_map2["location_code"] = _normalize_location_code(df_map2["location_code"])
    df_map2 = df_map2[df_map2["level_type"].str.upper().isin([t.upper() for t in level_types])]

    target = (df_map2[df_map2["location_code"].isin(set(sci_codes))]
              .drop_duplicates(subset=["location_code", "level_type"])
              .reset_index(drop=True))

    print(f"[INFO] Target codes selezionati: {len(target):,} (tipi: {sorted(target['level_type'].unique())})")
    return target

# ---------- 1) Letture geografiche + representative point ----------
def _representative_points(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.to_crs(4326)
    gdf["geometry"] = gdf["geometry"].representative_point()
    gdf["latitude"] = gdf.geometry.y
    gdf["longitude"] = gdf.geometry.x
    return gdf

# NUTS3
def load_nuts3_points(
    nuts_geojson_path: str,
    code_col: Optional[str] = None
) -> pd.DataFrame:
    """
    Legge il GeoJSON NUTS e restituisce (code, lat, lon) per LEVL 3.
    Prova a indovinare la colonna codice: NUTS_ID / NUTS_ID_ / id.
    """
    gdf = gpd.read_file(nuts_geojson_path)
    # filtra livello 3 se presente
    levl_col = next((c for c in ["LEVL_CODE", "LEVL", "LEVEL"] if c in gdf.columns), None)
    if levl_col is not None:
        gdf = gdf[gdf[levl_col].astype(str).isin(["3", 3])].copy()

    # determina colonna codice
    candidates = [code_col] if code_col else None
    if not candidates or candidates == [None]:
        candidates = [c for c in ["NUTS_ID", "nuts_id", "ID", "id"] if c in gdf.columns]
    if not candidates:
        raise ValueError("Non trovo una colonna codice per NUTS (es. 'NUTS_ID'). Passa code_col=...")

    cc = candidates[0]
    gdf = gdf.rename(columns={cc: "code"})
    gdf = _representative_points(gdf[["code", "geometry"]].dropna())
    gdf["code"] = _normalize_location_code(gdf["code"])
    return gdf[["code", "latitude", "longitude"]].drop_duplicates("code")

# GADM2 (GADM v4.10 GPKG)
def load_gadm2_points(gadm_gpkg_path: str, layer: str = "gadm_410", code_col: str = "GID_2"):
    """
    Estrae GADM livello 2 come (code, lat, lon) dal file GADM v4.10.
    - Usa il layer 'gadm_410' (che contiene tutti i livelli)
    - Seleziona la colonna GID_2 come identificativo ADM2
    - Filtra le righe dove GID_2 non è NaN
    - Calcola un punto rappresentativo all'interno di ciascun poligono
    """
    gdf = gpd.read_file(gadm_gpkg_path, layer=layer)

    if code_col not in gdf.columns:
        raise ValueError(f"La colonna {code_col!r} non esiste nel layer {layer}. Colonne trovate: {list(gdf.columns)}")

    # tieni solo livello 2
    gdf2 = gdf[~gdf[code_col].isna()].copy()
    gdf2 = gdf2.rename(columns={code_col: "code"})

    # calcolo punto rappresentativo
    gdf2 = gdf2.to_crs(4326)
    gdf2["geometry"] = gdf2.geometry.representative_point()
    gdf2["latitude"] = gdf2.geometry.y
    gdf2["longitude"] = gdf2.geometry.x

    # normalizza codice
    gdf2["code"] = gdf2["code"].astype("string").str.strip().str.upper()

    out = gdf2[["code", "latitude", "longitude"]].drop_duplicates("code")
    print(f"[GADM2] Caricate {len(out):,} unità ADM2 uniche da {gadm_gpkg_path}")
    return out

# US COUNTIES
def load_us_counties_points(
    counties_geojson_path: str,
    code_col: Optional[str] = None
) -> pd.DataFrame:
    """
    Estrae punti per contee USA.
    Colonne codice comuni: 'GEOID', 'geoid', 'FIPS', 'fips'.
    """
    gdf = gpd.read_file(counties_geojson_path)
    candidates = [code_col] if code_col else None
    if not candidates or candidates == [None]:
        candidates = [c for c in ["GEOID", "geoid", "FIPS", "fips"] if c in gdf.columns]
    if not candidates:
        raise ValueError("Non trovo una colonna codice per US counties (es. 'GEOID'). Passa code_col=...")

    cc = candidates[0]
    gdf = gdf.rename(columns={cc: "code"})
    # normalizza: GEOID/FIPS come stringa zero-padded (5)
    gdf["code"] = gdf["code"].astype(str).str.zfill(5)
    gdf = _representative_points(gdf[["code", "geometry"]].dropna())
    gdf["code"] = _normalize_location_code(gdf["code"])
    return gdf[["code", "latitude", "longitude"]].drop_duplicates("code")

# ---------- 2) Join: tieni SOLO i codici target (presenti in SCI + mapping) ----------
def build_nodes_for_type(
    target_codes: pd.DataFrame,
    type_name: str,
    source_df_points: pd.DataFrame
) -> pd.DataFrame:
    """
    Mantiene solo i codici di 'type_name' presenti in target_codes
    e con un match nei punti (source_df_points).
    Ritorna: nodeLabel, latitude, longitude (+ type).
    """
    type_mask = target_codes["level_type"].str.upper() == type_name.upper()
    wanted = target_codes.loc[type_mask, ["location_code"]].rename(columns={"location_code": "code"}).copy()
    wanted["code"] = _normalize_location_code(wanted["code"])

    pts = source_df_points.copy()
    pts["code"] = _normalize_location_code(pts["code"])

    out = wanted.merge(pts, on="code", how="inner")
    out = out.rename(columns={"code": "nodeLabel"})
    out["level_type"] = type_name.upper()
    return out[["nodeLabel", "latitude", "longitude", "level_type"]].drop_duplicates("nodeLabel")

# ---------- 3) Assemblaggio node_list finale ----------
def assemble_node_list(
    nuts_nodes: Optional[pd.DataFrame] = None,
    gadm_nodes: Optional[pd.DataFrame] = None,
    county_nodes: Optional[pd.DataFrame] = None
) -> pd.DataFrame:
    """
    Concatena le tre sorgenti, assegna nodeID stabile per label.
    Output colonne richieste: nodeID,nodeLabel,latitude,longitude
    (se serve puoi conservare 'level_type' per debug).
    """
    frames = [df for df in [nuts_nodes, gadm_nodes, county_nodes] if df is not None and len(df)]
    if not frames:
        raise ValueError("Nessun nodo fornito.")

    nodes = pd.concat(frames, ignore_index=True).drop_duplicates("nodeLabel")
    nodes = nodes.sort_values("nodeLabel").reset_index(drop=True)
    nodes.insert(0, "nodeID", range(1, len(nodes) + 1))
    return nodes[["nodeID", "nodeLabel", "latitude", "longitude"]]

import re
import pandas as pd

def normalize_gadm2_mapping_code(code: str) -> str | None:
    """
    Converte codici mapping tipo 'BGD1_1' in 'BGD.1.1_1' (formato GADM v4 GID_2).
    Ritorna None se il formato non combacia.
    """
    if code is None or pd.isna(code):
        return None
    s = str(code).strip().upper()
    m = re.fullmatch(r'([A-Z]{3})(\d+)_([0-9]+)', s)
    if not m:
        return None
    iso3, adm1, adm2 = m.groups()
    return f"{iso3}.{int(adm1)}.{int(adm2)}_1"

def normalize_county_mapping_code(code: str) -> str | None:
    """
    Converte codici mapping tipo 'USA06091' in '06091' (GEOID a 5 cifre).
    Gestisce anche 'USA31039' ecc. Restituisce None se non riconosce.
    """
    if code is None or pd.isna(code):
        return None
    s = str(code).strip().upper()
    if s.startswith("USA"):
        s = s[3:]
    s = re.sub(r'\D', '', s)  # lascia solo cifre
    if len(s) == 5:
        return s
    # se lunghezze diverse, prova a zfill
    if 1 <= len(s) <= 5:
        return s.zfill(5)
    return None

def build_nodes_for_type_with_transform(
    target_codes: pd.DataFrame,
    type_name: str,
    source_df_points: pd.DataFrame,
    transform_fn=None
) -> pd.DataFrame:
    """
    Come build_nodes_for_type, ma permette di trasformare i codici del mapping
    prima del join coi 'points' (es. GADM2 'BGD1_1' -> 'BGD.1.1_1').
    """
    mask = target_codes["level_type"].str.upper() == type_name.upper()
    wanted = target_codes.loc[mask, ["location_code"]].copy()
    wanted["location_code"] = wanted["location_code"].astype("string").str.strip().str.upper()

    if transform_fn is not None:
        wanted["code"] = wanted["location_code"].map(transform_fn)
    else:
        wanted["code"] = wanted["location_code"]

    # rimuovi codici non trasformabili
    wanted = wanted.dropna(subset=["code"]).drop_duplicates("code")

    pts = source_df_points.copy()
    pts["code"] = pts["code"].astype("string").str.strip().str.upper()

    out = wanted.merge(pts, on="code", how="inner")
    out = out.rename(columns={"code": "nodeLabel"})
    out["level_type"] = type_name.upper()
    return out[["nodeLabel", "latitude", "longitude", "level_type"]].drop_duplicates("nodeLabel")



In [26]:
import fiona

gpkg_path = "data/gadm_410.gpkg"  # metti qui il tuo percorso

layers = fiona.listlayers(gpkg_path)

print(f"Layer trovati in {gpkg_path}:")
for i, lyr in enumerate(layers, 1):
    print(f"{i:>2}. {lyr}")



Layer trovati in data/gadm_410.gpkg:
 1. gadm_410


In [27]:
import geopandas as gpd

nuts_path = "data/NUTS_RG_60M_2016_4326_LEVL_3.geojson"

# carico solo poche righe
nuts_sample = gpd.read_file(nuts_path, rows=5)

print("Colonne disponibili in NUTS3 GeoJSON:")
print(list(nuts_sample.columns))
print("\nPrime righe:")
print(nuts_sample.head())


Colonne disponibili in NUTS3 GeoJSON:
['LEVL_CODE', 'NUTS_ID', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME', 'MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'geometry']

Prime righe:
   LEVL_CODE NUTS_ID CNTR_CODE             NAME_LATN             NUTS_NAME  MOUNT_TYPE  URBN_TYPE  COAST_TYPE  \
0          3   CZ052        CZ  Královéhradecký kraj  Královéhradecký kraj           4          2           3   
1          3   CZ053        CZ       Pardubický kraj       Pardubický kraj           4          3           3   
2          3   CZ063        CZ         Kraj Vysočina         Kraj Vysočina           4          3           3   
3          3   CZ064        CZ     Jihomoravský kraj     Jihomoravský kraj           4          2           3   
4          3   CZ071        CZ        Olomoucký kraj        Olomoucký kraj           2          2           3   

                                            geometry  
0  POLYGON ((16.10732 50.66207, 16.33255 50.59246...  
1  POLYGON ((16.8042 49.59881, 16.39363 49

In [28]:
counties_path = "data/us-county-boundaries.geojson"

counties_sample = gpd.read_file(counties_path, rows=5)

print("Colonne disponibili in US counties GeoJSON:")
print(list(counties_sample.columns))
print("\nPrime righe:")
print(counties_sample.head())


Skipping field geo_point_2d: unsupported OGR type: 3


Colonne disponibili in US counties GeoJSON:
['intptlat', 'countyfp_nozero', 'countyns', 'stusab', 'csafp', 'state_name', 'aland', 'geoid', 'namelsad', 'countyfp', 'awater', 'classfp', 'lsad', 'name', 'funcstat', 'metdivfp', 'cbsafp', 'intptlon', 'statefp', 'mtfcc', 'geometry']

Prime righe:
      intptlat countyfp_nozero  countyns stusab csafp    state_name        aland  geoid          namelsad countyfp  \
0  +43.0066030             135  01266975     SD  None  South Dakota   1349873585  46135    Yankton County      135   
1  +41.5929185              49  00277289     CA  None    California  10225096402  06049      Modoc County      049   
2  +32.2388026             235  00347593     GA  None       Georgia    645583957  13235    Pulaski County      235   
3  +39.1642619              13  00424208     IL   476      Illinois    657422422  17013    Calhoun County      013   
4  +30.2064437               5  00558403     LA  None     Louisiana    751259388  22005  Ascension Parish      005   


In [30]:
import geopandas as gpd

def inspect_gadm2(gpkg_path: str, layer: str = "gadm_410", prefer_code_cols=("GID_2","ID_2","GID2")):
    """
    Ispeziona il tuo GPKG GADM4 e mostra cosa c'è a livello 2.
    - Tenta di individuare la colonna codice ADM2 (priorità: GID_2).
    - Filtra le righe ADM2 (non-NaN nella colonna individuata).
    - Stampa info utili + anteprima.
    Ritorna (gdf_full, gdf_adm2, code_col).
    """
    gdf = gpd.read_file(gpkg_path, layer=layer)
    print(f"[INFO] Caricato layer='{layer}' con {len(gdf):,} feature")
    print(f"[INFO] CRS: {gdf.crs}")
    print("[INFO] Colonne disponibili:")
    print(list(gdf.columns))

    # 1) individua colonna codice ADM2
    cols_up = {c.upper(): c for c in gdf.columns}
    code_col = None
    for pref in prefer_code_cols:
        if pref in cols_up:
            code_col = cols_up[pref]
            break
    if code_col is None:
        # fallback: qualunque colonna che sembri un identificativo di livello 2
        candidates = [orig for up, orig in cols_up.items() if up.endswith("_2") or "GID" in up]
        if not candidates:
            raise ValueError("Non trovo una colonna codice per ADM2 (tipo 'GID_2' o 'ID_2').")
        code_col = candidates[0]

    print(f"[INFO] Colonna codice ADM2 individuata: {code_col!r}")

    # 2) filtra solo le righe ADM2 (codice non nullo)
    gdf_adm2 = gdf[~gdf[code_col].isna()].copy()
    print(f"[INFO] Righe ADM2 (non-NaN su {code_col}): {len(gdf_adm2):,}")

    # 3) statistiche veloci su ADM2
    # codici unici
    unique_codes = gdf_adm2[code_col].astype("string").str.strip().str.upper().nunique()
    print(f"[INFO] Codici ADM2 unici: {unique_codes:,}")

    # tipo geometria
    geom_types = gdf_adm2.geom_type.value_counts().to_dict()
    print(f"[INFO] Geometrie (conteggio per tipo): {geom_types}")

    # anteprima colonne tipiche se presenti
    candidates_name = [c for c in ["NAME_0","NAME_1","NAME_2","GID_0","GID_1","GID_2"] if c in gdf_adm2.columns]
    show_cols = [code_col] + candidates_name
    show_cols = [c for c in show_cols if c in gdf_adm2.columns]
    show_cols = list(dict.fromkeys(show_cols))  # dedupe e preserva ordine

    print("\n== Anteprima ADM2 (prime 5 righe) ==")
    display(gdf_adm2[show_cols].head())

    # esempi di codici
    print("\n== Esempi di codici ADM2 ==")
    display(
        gdf_adm2[[code_col]]
        .astype("string").dropna()
        .drop_duplicates()
        .head(10)
    )

    # missing per alcune colonne utili (se esistono)
    check_cols = [c for c in ["NAME_0","NAME_1","NAME_2","GID_0","GID_1","GID_2"] if c in gdf_adm2.columns]
    if check_cols:
        print("\n== Missing count su colonne comuni ==")
        display(gdf_adm2[check_cols].isna().sum().to_frame("missing"))

    return gdf, gdf_adm2, code_col


In [31]:
gdf_full, gdf_adm2, gadm_code_col = inspect_gadm2("data/gadm_410.gpkg", layer="gadm_410")


[INFO] Caricato layer='gadm_410' con 356,508 feature
[INFO] CRS: EPSG:4326
[INFO] Colonne disponibili:
['UID', 'GID_0', 'NAME_0', 'VARNAME_0', 'GID_1', 'NAME_1', 'VARNAME_1', 'NL_NAME_1', 'ISO_1', 'HASC_1', 'CC_1', 'TYPE_1', 'ENGTYPE_1', 'VALIDFR_1', 'GID_2', 'NAME_2', 'VARNAME_2', 'NL_NAME_2', 'HASC_2', 'CC_2', 'TYPE_2', 'ENGTYPE_2', 'VALIDFR_2', 'GID_3', 'NAME_3', 'VARNAME_3', 'NL_NAME_3', 'HASC_3', 'CC_3', 'TYPE_3', 'ENGTYPE_3', 'VALIDFR_3', 'GID_4', 'NAME_4', 'VARNAME_4', 'CC_4', 'TYPE_4', 'ENGTYPE_4', 'VALIDFR_4', 'GID_5', 'NAME_5', 'CC_5', 'TYPE_5', 'ENGTYPE_5', 'GOVERNEDBY', 'SOVEREIGN', 'DISPUTEDBY', 'REGION', 'VARREGION', 'COUNTRY', 'CONTINENT', 'SUBCONT', 'geometry']
[INFO] Colonna codice ADM2 individuata: 'GID_2'
[INFO] Righe ADM2 (non-NaN su GID_2): 356,508
[INFO] Codici ADM2 unici: 47,218
[INFO] Geometrie (conteggio per tipo): {'MultiPolygon': 356508}

== Anteprima ADM2 (prime 5 righe) ==


,GID_2,NAME_0,NAME_1,NAME_2,GID_0,GID_1
0,AFG.1.1_1,Afghanistan,Badakhshan,Baharak,AFG,AFG.1_1
1,AFG.1.2_1,Afghanistan,Badakhshan,Darwaz,AFG,AFG.1_1
2,AFG.1.3_1,Afghanistan,Badakhshan,Fayzabad,AFG,AFG.1_1
3,AFG.1.4_1,Afghanistan,Badakhshan,Ishkashim,AFG,AFG.1_1
4,AFG.1.5_1,Afghanistan,Badakhshan,Jurm,AFG,AFG.1_1



== Esempi di codici ADM2 ==


,GID_2
0,AFG.1.1_1
1,AFG.1.2_1
2,AFG.1.3_1
3,AFG.1.4_1
4,AFG.1.5_1
5,AFG.1.6_1
6,AFG.1.7_1
7,AFG.1.8_1
8,AFG.1.9_1
9,AFG.1.10_1



== Missing count su colonne comuni ==


,missing
NAME_0,0
NAME_1,0
NAME_2,0
GID_0,0
GID_1,0
GID_2,0


In [48]:
# ---------- 0) Restrizione ai codici presenti in SCI + mapping ----------
target = select_target_codes(
    df_sci=df_sci,
    df_map=df_map,                # deve avere 'location_code' e 'level_type'
    level_types=("NUTS3", "GADM2", "COUNTY")
)

[INFO] Target codes selezionati: 6,117 (tipi: ['COUNTY', 'GADM2', 'NUTS3'])


In [49]:
# ---------- 1) Carico punti per ciascun tipo ----------
# NUTS3
nuts_pts = load_nuts3_points(
    nuts_geojson_path="data/NUTS_RG_60M_2016_4326_LEVL_3.geojson", 
    # code_col="NUTS_ID"  # di default prova a trovarlo da solo
)
print(nuts_pts.head(), len(nuts_pts))


    code   latitude  longitude
0  CZ052  50.321427  15.885255
1  CZ053  49.985521  16.154792
2  CZ063  49.411861  15.657920
3  CZ064  49.186884  16.707571
4  CZ071  49.861124  17.031093 1522


In [50]:
# US counties
county_pts = load_us_counties_points(
    counties_geojson_path="data/us-county-boundaries.geojson",
    # code_col="GEOID"    # se il file usa 'geoid' o 'FIPS', specifica qui
)
print(county_pts.head(), len(county_pts))

Skipping field geo_point_2d: unsupported OGR type: 3


    code   latitude   longitude
0  46135  42.983868  -97.398039
1  06049  41.590862 -120.723734
2  13235  32.254118  -83.487584
3  17013  39.134326  -90.655446
4  22005  30.204743  -90.891998 3233


In [51]:
# GADM2 (GPKG 4.10) – se fallisce, prova a cambiare 'layer' o 'code_col'
gadm_pts = load_gadm2_points(
    gadm_gpkg_path="data/gadm_410.gpkg",
    # layer="ADM_2",      # se serve: "ADM_ADM_2" / "ADM_2" a seconda del file
    # code_col="GID_2"    # tipico per GADM v4
)
print(gadm_pts.head(), len(gadm_pts))

[GADM2] Caricate 47,218 unità ADM2 uniche da data/gadm_410.gpkg
        code   latitude  longitude
0  AFG.1.1_1  36.941952  71.128464
1  AFG.1.2_1  38.221411  70.953644
2  AFG.1.3_1  37.101351  70.442711
3  AFG.1.4_1  36.864491  71.429559
4  AFG.1.5_1  36.621590  70.865658 47218


In [52]:
# GADM2: mapping BGD1_1 -> GADM GID_2 (BGD.1.1_1)
gadm_nodes = build_nodes_for_type_with_transform(
    target_codes=target,
    type_name="GADM2",
    source_df_points=gadm_pts,                 # ottenuto da load_gadm2_points(...)
    transform_fn=normalize_gadm2_mapping_code  # << trasformazione chiave
)

# COUNTY: mapping USA06091 -> GEOID '06091'
county_nodes = build_nodes_for_type_with_transform(
    target_codes=target,
    type_name="COUNTY",
    source_df_points=county_pts,                # ottenuto da load_us_counties_points(...)
    transform_fn=normalize_county_mapping_code  # << trasformazione chiave
)

# NUTS3 rimane invariato (già combacia con NUTS_ID)
nuts_nodes = build_nodes_for_type_with_transform(
    target_codes=target,
    type_name="NUTS3",
    source_df_points=nuts_pts,
    transform_fn=None
)

print("NUTS3 selezionati:", len(nuts_nodes))
print("GADM2 selezionati:", len(gadm_nodes))
print("COUNTY selezionati:", len(county_nodes))


NUTS3 selezionati: 1522
GADM2 selezionati: 82
COUNTY selezionati: 3225


In [53]:
# ---------- 2) Tieni SOLO i nodi apparsi in SCI + mapping ----------
nuts_nodes   = build_nodes_for_type(target, "NUTS3", nuts_pts)
gadm_nodes   = build_nodes_for_type(target, "GADM2", gadm_pts)
county_nodes = build_nodes_for_type(target, "COUNTY", county_pts)

print("NUTS3 selezionati:", len(nuts_nodes))
print("GADM2 selezionati:", len(gadm_nodes))
print("COUNTY selezionati:", len(county_nodes))

node_list = assemble_node_list(nuts_nodes, gadm_nodes, county_nodes)
display(node_list.head())
print(f"Totale nodi: {len(node_list):,}")
node_list.to_csv("node_list.csv", index=False)


# Salva il file nel formato richiesto
node_list.to_csv("node_list.csv", index=False)


NUTS3 selezionati: 1522
GADM2 selezionati: 0
COUNTY selezionati: 0


,nodeID,nodeLabel,latitude,longitude
0,1,AL011,41.657501,20.253305
1,2,AL012,41.390292,19.623216
2,3,AL013,42.160768,20.294509
3,4,AL014,41.786721,19.820903
4,5,AL015,42.200779,19.724310


Totale nodi: 1,522


In [41]:
df_map[df_map["level_type"].str.upper() == "GADM2"].head(20)


,location_code,level_type
6638,BGD1_1,GADM2
6639,BGD1_2,GADM2
6640,BGD1_3,GADM2
6641,BGD1_4,GADM2
6642,BGD1_5,GADM2
6643,BGD1_6,GADM2
6644,BGD2_7,GADM2
6645,BGD2_8,GADM2
6646,BGD2_9,GADM2
6647,BGD2_10,GADM2


In [42]:
df_map[df_map["level_type"].str.upper() == "COUNTY"].head(20)


,location_code,level_type
3409,USA31039,COUNTY
3410,USA53069,COUNTY
3411,USA35011,COUNTY
3412,USA31109,COUNTY
3413,USA31129,COUNTY
3414,USA72085,COUNTY
3415,USA46099,COUNTY
3416,USA48327,COUNTY
3417,USA06091,COUNTY
3418,USA21053,COUNTY
